In [1]:
#import required packages
import nltk
import json
import random
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer

In [2]:
mod = load_model('uofmchatbot_model')
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
all_words = pickle.load(open('all_words.pkl','rb'))
all_classes = pickle.load(open('all_classes.pkl','rb'))

In [3]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [4]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, all_words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(all_words)  
    for s in sentence_words:
        for i,w in enumerate(all_words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [5]:
def predict_class(sentence, mod):
    # filter out predictions below a threshold
    p = bow(sentence, all_words,show_details=False)
    res = mod.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": all_classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(msg):
    ints = predict_class(msg, mod)
    res = getResponse(ints, intents)
    return res

In [20]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#723845", font=("calibri", 14 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Tigerbot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Tigerbot")
base.geometry("500x600")
base.resizable(width=50, height=500)

#Create Chat window
ChatLog = Text(base, bd=1, bg="#ADD8E6", height="10", width="6", font="calibri",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("calibri",14,'bold'), text="Send", width="12", height=6,
                    bd=0, bg="#0000FF", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="calibri")


#Place all components on the screen
scrollbar.place(x=400,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=410)
EntryBox.place(x=150, y=401, height=90, width=250)
SendButton.place(x=7, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 52ms/step
